In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial import distance
import xgboost as xgb
import shap

/Users/herminetranie/opt/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ethnicity = pd.read_csv('IPEDS_Dataset_Encoded/Fall Enrollment/EFA_2015-2020_data.csv', sep=',')

In [3]:
def JSD(b): 
    '''
    Jensen-Shannon distance between two vectors
    '''
    a = overall_ethn
    b = np.array(b, dtype=float)
    _M = 0.5 * (a + b)
    return distance.jensenshannon(a,b)

ethn = ethnicity[(ethnicity.efalevel == 'All students total')].set_index('unitid')
ethn.loc[:,'other'] = ethn[['ef2mort',"efunknt","efunknt"]].sum(axis=1)

data_ethn = ethn[['efaiant','efasiat','efbkaat','efhispt','efnhpit','efwhitt','other']].div(ethn['eftotlt'], axis=0)
overall_ethn = np.array(data_ethn.mean())

# save overall_ethn to a file
np.savetxt('overall_ethn.csv', overall_ethn, delimiter=',')

In [4]:
data_with_level = pd.read_csv('data_with_level.csv')

In [5]:
list_schools = np.array(data_with_level['unitid'])
dico_school_cdf_fee = data_with_level[['unitid','cdf_fee']].set_index('unitid').to_dict()['cdf_fee']
dico_school_level = data_with_level[['unitid','level']].set_index('unitid').to_dict()['level']
dico_school_grad_rate = data_with_level[['unitid','omawdp4']].set_index('unitid').to_dict()['omawdp4']
#omawdp4

def parametric_institution_quality_school(school, p):
    p = np.ndarray.flatten(np.array(p))
    diversity_score = JSD(p)
    #graduation_rate = model_grad_rate.predict([p])[0]
    graduation_rate = dico_school_grad_rate[school]
    fee_cdf = dico_school_cdf_fee[school]
    #admission_rate = model_level.predict([p])[0]
    admission_rate = dico_school_level[school]
    return (diversity_score*graduation_rate)/(fee_cdf*admission_rate)

def parametric_institution_quality(p_list):
    df = data_with_level.copy()
    # flatten p_list
    #np.ndarray.flatten(np.array(p_list))
    #df[['efaiant', 'efasiat', 'efbkaat', 'efhispt', 'efnhpit','efwhitt', 'other']]=p_list
    df['school_quality']=df[['unitid','efaiant', 'efasiat', 'efbkaat', 'efhispt', 'efnhpit','efwhitt', 'other']].apply(
    lambda x: parametric_institution_quality_school(x[0],[x[i] for i in range(1,8)]), axis=1)
    #x[0],[x[i] for i in range(1,8)])
    return df['school_quality'].mean()

def param_quality(p):
    #print("p", p)
    #print("shape of p is", p.shape)
    #p_list = p.reshape(len(data_with_level),7)
    #print("p_list", p_list)
    #print("shape of p_list is", p_list.shape)
    return -parametric_institution_quality(p)

cons = ({'type': 'eq',
         'fun' : lambda x: x[7*i+1]+x[7*i+2]+x[7*i+3]+x[7*i+4]+x[7*i+5]+x[7*i+6] - 1
        } for i in range(len(data_with_level)))


initial_guess = np.ndarray.flatten(
    np.array(data_with_level[['efaiant', 'efasiat', 'efbkaat', 'efhispt', 'efnhpit','efwhitt', 'other']])
                                    )

In [6]:
# use gurobi to solve the problem and display the results
import gurobipy as gp
from gurobipy import GRB
import sys
import numpy as np
import pandas as pd
import time

# set the maximum time for the solver to run (in seconds)
max_time = 60

# create a new model for version 9.0 of gurobi
m = gp.Model()

# create variables
x = m.addVars(len(data_with_level), 7, vtype=GRB.CONTINUOUS, name="x")

# define starting point as the initial guess
m.setParam('Start', initial_guess)

# set objective as param_quality function
m.setObjective(param_quality(x), GRB.MAXIMIZE)

# add constraints as the cons dictionary defined above
m.addConstrs((x.sum(i,'*') == 1 for i in range(len(data_with_level))), name="c")


#for i in range(len(data_with_level)):
    #m.addConstr(gp.quicksum(x[i,j] for j in range(7)) == 1)

# optimize model
m.optimize()

# print solution even if not optimal
if m.status == GRB.Status.OPTIMAL:
    print('Optimal objective: %g' % m.objVal)
#else:
    #print('Optimization ended with status %d' % m.status)

#get solution
sol = m.getAttr('x', x)
sol = np.array(list(sol.values()))
sol = sol.reshape(len(data_with_level),7)
print(sol.shape)
print(sol)




Academic license - for non-commercial use only - expires 2023-10-30
Using license file /Users/herminetranie/gurobi.lic
No parameters matching 'Start' found
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64[arm])
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads
Optimize a model with 1492 rows, 10444 columns and 10444 nonzeros
Model fingerprint: 0xdac0ed88
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 1492 rows and 10444 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.9420359e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective -1.942035891e+00
Optimal objective: -1.94204
(1492, 7)
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0

invalid value encountered in double_scalars


In [16]:
param_quality(initial_guess)

p [0.00233418 0.00458423 0.43961975 ... 0.00081277 0.63634542 0.08335931]
shape of p is (10444,)
p_list [[2.33418041e-03 4.58422624e-03 4.39619749e-01 ... 2.47331326e-03
  2.58412309e-01 5.52461215e-01]
 [5.20894803e-03 3.74447924e-02 1.28224484e-01 ... 0.00000000e+00
  7.85661128e-01 1.25406234e-02]
 [0.00000000e+00 6.84462697e-04 9.19828775e-01 ... 0.00000000e+00
  3.40591091e-02 0.00000000e+00]
 ...
 [0.00000000e+00 2.50000000e-02 0.00000000e+00 ... 0.00000000e+00
  5.25000000e-01 0.00000000e+00]
 [9.25000000e-01 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  7.50000000e-02 0.00000000e+00]
 [1.31379710e-03 6.55228467e-02 5.59922508e-02 ... 8.12772780e-04
  6.36345417e-01 8.33593124e-02]]
shape of p_list is (1492, 7)


NameError: name 'JSD' is not defined

In [32]:
# force the minimization to stop after 60 seconds
# this is not guaranteed to find the global minimum
# but it will find a local minimum in a reasonable amount of time

from scipy.optimize import minimize
import time
start_time = time.time()
res = minimize(param_quality, initial_guess, method='SLSQP', constraints=cons, options={'maxiter': 1000})
print("--- %s seconds ---" % (time.time() - start_time))


invalid value encountered in double_scalars
invalid value encountered in double_scalars
invalid value encountered in double_scalars
invalid value encountered in double_scalars
invalid value encountered in double_scalars
invalid value encountered in double_scalars
invalid value encountered in double_scalars
invalid value encountered in double_scalars
invalid value encountered in double_scalars
invalid value encountered in double_scalars
invalid value encountered in double_scalars
invalid value encountered in double_scalars
invalid value encountered in double_scalars
invalid value encountered in double_scalars
invalid value encountered in double_scalars
invalid value encountered in double_scalars
invalid value encountered in double_scalars
invalid value encountered in double_scalars
invalid value encountered in double_scalars
invalid value encountered in double_scalars
invalid value encountered in double_scalars
invalid value encountered in double_scalars
invalid value encountered in dou

KeyboardInterrupt: 

In [28]:
from scipy.optimize import minimize

nb_eval = 1
def callbackF(x):
    global nb_eval
    print(nb_eval, param_quality(x))
    nb_eval += 1
    
opt_ethn_prop = minimize(param_quality, initial_guess, constraints = cons, options={'maxiter':5}, callback=callbackF)


`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
invalid value encountered in double_scalars
`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
invalid value encountered in double_scalars
`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar t

KeyboardInterrupt: 